# stream

In [ ]:
// // test

open testing

()



In [ ]:
union rec stream t =
    | StreamCons : t * (() -> stream t)
    | StreamNil

()



## fold

In [ ]:
inl fold fn init s =
    inl rec body acc = function
        | StreamCons (st, fn') => loop (fn acc st) (fn' ())
        | StreamNil => acc
    and inl loop acc = loop_body body acc
    loop init s

()



## fold_back

In [ ]:
inl fold_back fn s init =
    inl rec body acc = function
        | StreamCons (st, fn') => fn st (loop acc (fn' ()))
        | StreamNil => acc
    and inl loop acc = loop_body body acc
    loop init s

()



## to_list

In [ ]:
inl to_list s =
    (s, [])
    ||> fold_back fun x acc =>
        x :: acc

()



## rev

In [ ]:
inl rev s =
    (StreamNil, s)
    ||> fold fun s x =>
        StreamCons (x, fun () => s)

()



## from_list

In [ ]:
inl from_list list =
    (list, StreamNil)
    ||> listm.foldBack fun x acc =>
        StreamCons (x, fun () => acc)

()



In [ ]:
// // test

listm.init 3i32 id
|> from_list
|> rev
|> to_list
|> _equal [ 2; 1; 0 ]

type UH0 =
    | UH0_0 of int32 * UH0
    | UH0_1
let rec method0 () : unit =
    let v18 : UH0 = UH0_1
    let v19 : UH0 = UH0_0(0, v18)
    let v20 : UH0 = UH0_0(1, v19)
    let v21 : UH0 = UH0_0(2, v20)
    let v22 : UH0 = UH0_1
    let v23 : UH0 = UH0_0(0, v22)
    let v24 : UH0 = UH0_0(1, v23)
    let v25 : UH0 = UH0_0(2, v24)
    let v26 : string = $"_equal / actual: %A{v21} / expected: %A{v25}"
    ()
method0()



## try_item

In [ ]:
inl try_item i s =
    inl rec body i = function
        | StreamCons (x, _) when i = 0 => Some x
        | StreamCons (_, fn) => loop (i - 1) (fn ())
        | StreamNil => None
    and inl loop acc = loop_body body acc
    loop i s

inl item i =
    try_item i >> optionm.value

()



In [ ]:
// // test

listm.init 10i32 id
|> from_list
|> item 9i32
|> _equal 9

let rec method0 () : unit =
    let v0 : string = $"_equal / actual: %A{9} / expected: %A{9}"
    ()
method0()



## new_infinite_stream

In [ ]:
inl new_infinite_stream fn =
    inl rec loop n =
        StreamCons (fn n, fun () => loop (n + 1))
    loop 0

inl new_infinite_stream_ fn =
    let rec loop n =
        StreamCons (fn n, fun () => loop (n + 1))
    loop 0

()



In [ ]:
// // test

new_infinite_stream print_and_return
|> item 4i32
|> _equal 4i32

let rec method0 () : unit =
    printfn $"print_and_return / x: {0}"
    printfn $"print_and_return / x: {1}"
    printfn $"print_and_return / x: {2}"
    printfn $"print_and_return / x: {3}"
    printfn $"print_and_return / x: {4}"
    let v0 : string = $"_equal / actual: %A{4} / expected: %A{4}"
    ()
method0()

print_and_return / x: 0
print_and_return / x: 1
print_and_return / x: 2
print_and_return / x: 3
print_and_return / x: 4


## new_finite_stream

In [ ]:
inl new_finite_stream fn max =
    inl rec loop n =
        if n >= max
        then StreamNil
        else StreamCons (fn n, fun () => loop (n + 1))
    loop 0

()



## memoize

In [ ]:
union memoized_stream t =
    | NotComputed : () -> stream t
    | Computed : stream t

inl memoize s =
    inl rec body () s =
        inl state = mut (NotComputed s)
        fun () =>
            match *state with
            | Computed x => x
            | NotComputed fn =>
                inl new_state =
                    match fn () with
                    | StreamNil => StreamNil
                    | StreamCons (x, fn) => StreamCons (x, loop fn)
                state <- Computed new_state
                new_state
    and inl loop s = loop_body body () s
    loop (fun () => s)

()



In [ ]:
// // test

inl memo_stream = new_finite_stream print_and_return 10 |> memoize

memo_stream ()
|> item 3i32
|> _equal 3i32

memo_stream ()
|> item 5i32
|> _equal 5i32

type UH0 =
    | UH0_0 of int32 * (unit -> UH0)
    | UH0_1
and [<Struct>] US0 =
    | US0_0 of f0_0 : UH0
    | US0_1 of f1_0 : (unit -> UH0)
and Mut0 = {mutable l0 : US0}
and [<Struct>] US1 =
    | US1_0
    | US1_1 of f1_0 : int32
let rec closure10 () () : UH0 =
    UH0_1
and closure9 () () : UH0 =
    printfn $"print_and_return / x: {9}"
    let v0 : (unit -> UH0) = closure10()
    UH0_0(9, v0)
and closure8 () () : UH0 =
    printfn $"print_and_return / x: {8}"
    let v0 : (unit -> UH0) = closure9()
    UH0_0(8, v0)
and closure7 () () : UH0 =
    printfn $"print_and_return / x: {7}"
    let v0 : (unit -> UH0) = closure8()
    UH0_0(7, v0)
and closure6 () () : UH0 =
    printfn $"print_and_return / x: {6}"
    let v0 : (unit -> UH0) = closure7()
    UH0_0(6, v0)
and closure5 () () : UH0 =
    printfn $"print_and_return / x: {5}"
    let v0 : (unit -> UH0) = closure6()
    UH0_0(5, v0)
and closure4 () () : UH0 =
    printfn $"print_and_return / x: {4}"
    let v0 : (unit -> UH0) = c

In [ ]:
// // test

inl memo_stream = new_infinite_stream_ print_and_return |> memoize

memo_stream ()
|> item 3i32
|> _equal 3i32

memo_stream ()
|> item 5i32
|> _equal 5i32

type UH0 =
    | UH0_0 of int32 * (unit -> UH0)
    | UH0_1
and [<Struct>] US0 =
    | US0_0 of f0_0 : UH0
    | US0_1 of f1_0 : (unit -> UH0)
and Mut0 = {mutable l0 : US0}
and [<Struct>] US1 =
    | US1_0
    | US1_1 of f1_0 : int32
let rec closure0 (v0 : int32) () : UH0 =
    let v1 : int32 = v0 + 1
    method1(v1)
and method1 (v0 : int32) : UH0 =
    printfn $"print_and_return / x: {v0}"
    let v1 : (unit -> UH0) = closure0(v0)
    UH0_0(v0, v1)
and closure1 (v0 : UH0) () : UH0 =
    v0
and closure2 (v0 : Mut0) () : UH0 =
    let v1 : US0 = v0.l0
    match v1 with
    | US0_0(v2) -> (* Computed *)
        v2
    | US0_1(v3) -> (* NotComputed *)
        let v4 : UH0 = v3 ()
        let v11 : UH0 =
            match v4 with
            | UH0_0(v6, v7) -> (* StreamCons *)
                let v8 : (unit -> UH0) = method2(v7)
                UH0_0(v6, v8)
            | UH0_1 -> (* StreamNil *)
                UH0_1
        let v12 : US0 = US0_0(v11)
        v0.l0 <- v12
        v11
and 

## unfold

In [ ]:
inl unfold f x0 =
    inl rec body () x =
        match f x with
        | Some (x', y) => StreamCons (x', fun () => loop y)
        | None => StreamNil
    and inl loop x = loop_body body () x
    loop x0

()



## iterate

In [ ]:
inl iterate f =
    unfold (fun x => Some (x, f x))

()



In [ ]:
// // test

iterate ((*) 2) 1i32
|> item 10i32
|> _equal 1024

let rec method0 () : unit =
    let v0 : string = $"_equal / actual: %A{1024} / expected: %A{1024}"
    ()
method0()



## take_while

In [ ]:
inl take_while cond s =
    inl rec body i = function
        | StreamCons (st, fn) when cond st i => StreamCons (st, fun () => loop (i + 1) (fn ()))
        | _ => StreamNil
    and inl loop i = loop_body body i
    loop 0 s

()



## sum

In [ ]:
inl sum seq =
    seq |> fold (+) 0

()



In [ ]:
// // test

listm.init 10i32 id
|> from_list
|> sum
|> _equal 45

let rec method0 () : unit =
    let v0 : string = $"_equal / actual: %A{45} / expected: %A{45}"
    ()
method0()



In [ ]:
// // test

new_finite_stream print_and_return 10i32
|> take_while (fun n (_ : i32) => n < 5)
|> sum
|> _equal 10

let rec method0 () : unit =
    printfn $"print_and_return / x: {0}"
    printfn $"print_and_return / x: {1}"
    printfn $"print_and_return / x: {2}"
    printfn $"print_and_return / x: {3}"
    printfn $"print_and_return / x: {4}"
    printfn $"print_and_return / x: {5}"
    let v0 : string = $"_equal / actual: %A{10} / expected: %A{10}"
    ()
method0()

print_and_return / x: 0
print_and_return / x: 1
print_and_return / x: 2
print_and_return / x: 3
print_and_return / x: 4
print_and_return / x: 5


In [ ]:
// // test

new_infinite_stream print_and_return
|> take_while (fun n (_ : i32) => n < 5i32)
|> sum
|> _equal 10

let rec method0 () : unit =
    printfn $"print_and_return / x: {0}"
    printfn $"print_and_return / x: {1}"
    printfn $"print_and_return / x: {2}"
    printfn $"print_and_return / x: {3}"
    printfn $"print_and_return / x: {4}"
    printfn $"print_and_return / x: {5}"
    let v0 : string = $"_equal / actual: %A{10} / expected: %A{10}"
    ()
method0()

print_and_return / x: 0
print_and_return / x: 1
print_and_return / x: 2
print_and_return / x: 3
print_and_return / x: 4
print_and_return / x: 5


In [ ]:
// // test

iterate ((*) 6) 1i32
|> take_while (fun _ i => i <= 7i32)
|> to_list
|> _equal [ 1i32; 6; 36; 216; 1296; 7776; 46656; 279936 ]

type UH0 =
    | UH0_0 of int32 * UH0
    | UH0_1
let rec method0 () : unit =
    let v63 : UH0 = UH0_1
    let v64 : UH0 = UH0_0(279936, v63)
    let v65 : UH0 = UH0_0(46656, v64)
    let v66 : UH0 = UH0_0(7776, v65)
    let v67 : UH0 = UH0_0(1296, v66)
    let v68 : UH0 = UH0_0(216, v67)
    let v69 : UH0 = UH0_0(36, v68)
    let v70 : UH0 = UH0_0(6, v69)
    let v71 : UH0 = UH0_0(1, v70)
    let v72 : UH0 = UH0_1
    let v73 : UH0 = UH0_0(279936, v72)
    let v74 : UH0 = UH0_0(46656, v73)
    let v75 : UH0 = UH0_0(7776, v74)
    let v76 : UH0 = UH0_0(1296, v75)
    let v77 : UH0 = UH0_0(216, v76)
    let v78 : UH0 = UH0_0(36, v77)
    let v79 : UH0 = UH0_0(6, v78)
    let v80 : UH0 = UH0_0(1, v79)
    let v81 : string = $"_equal / actual: %A{v71} / expected: %A{v80}"
    ()
method0()



## indexed

In [ ]:
inl indexed s =
    ((StreamNil, 0), s)
    ||> fold fun (acc, i) x =>
        StreamCons ((i, x), fun () => acc), i + 1
    |> fst
    |> rev

()



In [ ]:
// // test

listm.init 10i32 ((*) 2)
|> from_list
|> indexed
|> item 5i32
|> _equal (5i32, 10i32)

let rec method0 () : unit =
    let v0 : string = $"_equal / actual: %A{struct (5, 10)} / expected: %A{struct (5, 10)}"
    ()
method0()



## map

In [ ]:
inl map fn s =
    (s, StreamNil)
    ||> fold_back fun x acc =>
        StreamCons (fn x, fun () => acc)

()



In [ ]:
// // test

listm.init 10i32 id
|> from_list
|> map ((*) 2)
|> item 5i32
|> _equal 10i32

let rec method0 () : unit =
    let v0 : string = $"_equal / actual: %A{10} / expected: %A{10}"
    ()
method0()



## zip_with

In [ ]:
inl zip_with fn s1 s2 =
    inl rec loop s1 s2 =
        match s1, s2 with
        | StreamCons (st1, fn1), StreamCons (st2, fn2) =>
            StreamCons (fn st1 st2, fun () => loop (fn1 ()) (fn2 ()))
        | StreamNil, _ | _, StreamNil => StreamNil
    loop s1 s2

inl zip_with_ fn s1 s2 =
    let rec loop s1 s2 =
        match s1, s2 with
        | StreamCons (st1, fn1), StreamCons (st2, fn2) =>
            StreamCons (fn st1 st2, fun () => loop (fn1 ()) (fn2 ()))
        | StreamNil, _ | _, StreamNil => StreamNil
    loop s1 s2

()



In [ ]:
// // test

((listm.init 10i32 id |> from_list), (listm.init 10i32 ((*) 2) |> from_list))
||> zip_with (+)
|> item 2i32
|> _equal 6

let rec method0 () : unit =
    let v0 : string = $"_equal / actual: %A{6} / expected: %A{6}"
    ()
method0()



## zip

In [ ]:
inl zip s1 s2 =
    zip_with pair s1 s2

inl zip_ s1 s2 =
    zip_with_ pair s1 s2

()



In [ ]:
// // test

((listm.init 10i32 id |> from_list), (listm.init 10i32 ((*) 2) |> from_list))
||> zip
|> item 5i32
|> _equal (5, 10)

let rec method0 () : unit =
    let v0 : string = $"_equal / actual: %A{struct (5, 10)} / expected: %A{struct (5, 10)}"
    ()
method0()



## unzip

In [ ]:
inl unzip s =
    inl rec loop s =
        match s with
        | StreamCons ((x, y), fn) =>
            inl xs, ys = loop (fn ())
            StreamCons (x, fun () => xs), StreamCons (y, fun () => ys)
        | StreamNil => pair StreamNil StreamNil
    loop s

()



In [ ]:
// // test

listm.init 10i32 id
|> listm.map (fun x => x, x)
|> from_list
|> unzip
|> fun x, y =>
    x |> sum
    |> _equal 45

    y |> sum
    |> _equal 45

let rec method0 () : unit =
    let v0 : string = $"_equal / actual: %A{45} / expected: %A{45}"
    let v1 : string = $"_equal / actual: %A{45} / expected: %A{45}"
    ()
method0()

